# Entanglement: Advanced 0

In previous tutorial, we have seen SEP set which consists of separable states, and ENT set which consists of entangled states. 

$$ \mathrm{SEP}=\mathrm{conv}\left(\mathcal{D}_{A}\otimes\mathcal{D}_{B}\right) \subseteq \mathcal{D}_{d_Ad_B} $$

$$ \mathrm{ENT}=\mathcal{D}_{d_{A}d_{B}}\setminus\mathrm{SEP} $$

where $\mathcal{D}_d$ denotes the set of density matrices on $\mathcal{H}_d$.

Besides checking whether a state is entangled or not, we can also ask the following questions

1. how far is a state from the set of separable states, theoretically, entanglement measure.
2. where is the boundary between separable states and entangled states.

In this tutorial we will focus on these two questions.

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numqi


## Entanglement measure

There are some entanglement measure [wiki-link](https://en.wikipedia.org/wiki/Quantum_entanglement#As_a_measure_of_entanglement)

1. concurrence
2. relative entropy of entanglement (REE)
3. entanglement of formation (EoF)
4. logarithmic negativity

These measures are zero for all SEP states, positive for ENT states, and non-decreasing under local operations and classical communication (LOCC) [quantiki-link](https://www.quantiki.org/wiki/entanglement-measure). In this tutorial we will focus on REE which is defined as [wiki-link](https://en.wikipedia.org/wiki/Quantum_relative_entropy)

$$ \mathrm{REE}\left( \rho \right) =\min_{\sigma \in \mathrm{SEP}} S\left( \rho ||\sigma \right) $$

where $S$ is the quantum relative entropy (von Neumann entropy) defined as

$$ S\left( \rho ||\sigma \right) =\mathrm{Tr}\left[ \rho \ln \rho -\rho \ln \sigma \right] $$

The relative entropy $S$ is jointly convex in its arguments, and non-negative. It is zero if and only if $\rho=\sigma$. In the following, we will discuss Werner state and Isotropic state which has analytical solution for REE.

### Werner state

Werner state is defined as [wiki-link](https://en.wikipedia.org/wiki/Werner_state) over $\mathcal{H}_d\otimes \mathcal{H}_d$

$$ \rho_{W,d}\left(\alpha\right)=\frac{1}{d^{2}-d\alpha}I-\frac{\alpha}{d^{2}-d\alpha}\sum_{ij}\left|ij\right\rangle \left\langle ji\right|,\quad\alpha\in\left[-1,1\right] $$

(You might see different definitions for Werner state, you can check [doc-link](./special_state.md#werner-state) for the conversion among different definitions)

Werner state is separable for $\alpha\in\left[-1,\frac{1}{d}\right]$, and entangled for $\alpha\in\left(\frac{1}{d},1\right]$. The REE of Werner state is given in paper "Entanglement measures under symmetry" [doi-link](https://doi.org/10.1103/PhysRevA.64.062307).

$$ \mathrm{REE\left(\rho_{W,d}(\alpha)\right)=\begin{cases}
0 & \alpha\in\left[-1,\frac{1}{d}\right]\\
S\left(\rho_{W,d}(\alpha)\lvert\rho_{W,d}\left(\frac{1}{d}\right)\right) & \alpha\in\left[\frac{1}{d},1\right]
\end{cases}} $$

Below, we can calculate the REE via PPT criterion (lower bound) and CHA criterion (upper bound), and compare with the analytical solution.

In [ ]:
dim = 3
alpha_list = np.linspace(0, 1, 100, endpoint=False) # alpha=1 is unstable for matrix logarithm
dm_target_list = [numqi.state.Werner(dim, x) for x in alpha_list]


PPT criterion is to solve the following semidefinite program (SDP). 

$$ \min_{\sigma}\;S\left(\rho_{W,d}\left(\alpha\right)\|\sigma\right) $$

$$ s.t.\;\begin{cases}
\rho\succeq0\\
\mathrm{Tr}\left[\rho\right]=1\\
\rho^{\Gamma}\succeq0
\end{cases} $$

It can be approximately evaluated by semidefinite programming (SDP) according to the paper "Semidefinite approximations of the matrix logarithm" [doi-link](https://doi.org/10.1007%2Fs10208-018-9385-0) [arxiv-link](https://arxiv.org/abs/1705.00812) [github/cvxquad](https://github.com/hfawzi/cvxquad).

In [ ]:
ree_analytical = np.array([numqi.state.get_Werner_ree(dim, x) for x in alpha_list])
ree_ppt = numqi.entangle.get_ppt_ree(dm_target_list, dim, dim) #about 30 seconds


Convex hull approximation (CHA) is to solve the following non-convex optimization problem via gradient descent method. The key idea is to formulize the problem as a non-convex optimization problem, and use gradient descent method to find the optimal solution. The gradient of the objective function is solved by gradient back-propagation method from Pytorch framework. We will dive into more details in the following tutorial.

In [ ]:
model = numqi.entangle.AutodiffCHAREE((dim,dim), distance_kind='ree')
ree_cha = [] #about 11 seconds
for dm_target_i in tqdm(dm_target_list):
    model.set_dm_target(dm_target_i)
    tmp0 = numqi.optimize.minimize(model, theta0='uniform', tol=1e-12, num_repeat=1, print_every_round=0).fun
    ree_cha.append(tmp0)
ree_cha = np.array(ree_cha)


In [ ]:
fig, ax = plt.subplots()
ax.plot(alpha_list, ree_analytical, "x", label="analytical")
ax.plot(alpha_list, ree_ppt, label="PPT")
ax.plot(alpha_list, ree_cha, label="CHA")
ax.set_xlim(min(alpha_list), max(alpha_list))
ax.set_ylim(1e-13, 1)
ax.set_yscale('log')
ax.set_xlabel(r"$\beta$")
ax.set_ylabel("REE")
ax.set_title(f'Werner({dim})')
ax.legend()
fig.tight_layout()


Be care, there is NOT a phase transition in the figure. the sharp shift is only due to the logarithmic scale used in the y-axis. We can see that the upper bound (CHA) and lower bound (PPT) are very close to the analytical solution.

### Isotropic state

Let's repeat the same procedure for Isotropic state. Isotropic state is defined as [quantiki-link](https://www.quantiki.org/wiki/isotropic-state) over $\mathcal{H}_d\otimes \mathcal{H}_d$

$$ \rho_{I,d}^{\prime}\left(\alpha\right)=\frac{1-\alpha}{d^{2}}I+\frac{\alpha}{d}\sum_{i}\left|ii\right\rangle \left\langle ii\right|,\quad\alpha\in\left[-\frac{1}{d^{2}-1},1\right] $$

(You might see different definitions for Werner state, you can check [doc-link](./special_state.md#isotropic-state) for the conversion among different definitions)

Isotropic state is separable for $\alpha\in\left[-\frac{1}{d^2-1},\frac{1}{d+1}\right]$, and entangled for $\alpha\in\left(\frac{1}{d+1},1\right]$. The REE of Werner state is given in paper "Bound on distillable entanglement" [doi-link](https://doi.org/10.1103/PhysRevA.60.179).

$$ \mathrm{REE\left(\rho_{I,d}(\alpha)\right)=\begin{cases}
0 & \alpha\in\left[-\frac{1}{d^{2}-1},\frac{1}{d+1}\right]\\
S\left(\rho_{I,d}(\alpha)\lvert\rho_{I,d}\left(\frac{1}{d+1}\right)\right) & \alpha\in\left[\frac{1}{d+1},1\right]
\end{cases}} $$

In [ ]:
dim = 3
alpha_list = np.linspace(0, 1, 100, endpoint=False) # alpha=1 is unstable for matrix logarithm
dm_target_list = [numqi.state.Isotropic(dim, x) for x in alpha_list]

ree_analytical = np.array([numqi.state.get_Isotropic_ree(dim, x) for x in alpha_list])
ree_ppt = numqi.entangle.get_ppt_ree(dm_target_list, dim, dim) #about 30 seconds

model = numqi.entangle.AutodiffCHAREE((dim, dim), distance_kind='ree')
ree_cha = [] #about 11 seconds
for dm_target_i in tqdm(dm_target_list):
    model.set_dm_target(dm_target_i)
    tmp0 = numqi.optimize.minimize(model, theta0='uniform', tol=1e-12, num_repeat=1, print_every_round=0).fun
    ree_cha.append(tmp0)
ree_cha = np.array(ree_cha)

fig, ax = plt.subplots()
ax.plot(alpha_list, ree_analytical, "x", label="analytical")
ax.plot(alpha_list, ree_ppt, label="PPT")
ax.plot(alpha_list, ree_cha, label="CHA")
ax.set_xlim(min(alpha_list), max(alpha_list))
ax.set_ylim(1e-13, 1)
ax.set_yscale('log')
ax.set_xlabel(r"$\beta$")
ax.set_ylabel("REE")
ax.legend()
ax.set_title(f"Isotropic({dim})")
fig.tight_layout()


## SEP Boundary

TODO